<a href="https://colab.research.google.com/github/plus2net/Python-basics/blob/main/Date_time_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://www.plus2net.com/images/top2.jpg)        More on  [Date and time data  ](https://www.plus2net.com/sql_tutorial/date-lastweek-dump.php)

Records for duration ( + and - )370 days from today.

In [ ]:
!wget -O dt_table.csv https://www.plus2net.com/sql_tutorial/dt_table.php

--2025-11-29 11:33:39--  https://www.plus2net.com/sql_tutorial/dt_table.php
Resolving www.plus2net.com (www.plus2net.com)... 184.168.96.1
Connecting to www.plus2net.com (www.plus2net.com)|184.168.96.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘dt_table.csv’

dt_table.csv            [ <=>                ]  17.54K   105KB/s    in 0.2s    

2025-11-29 11:33:40 (105 KB/s) - ‘dt_table.csv’ saved [17962]



Records for query in the range of previous 12 months to future 3 months

In [ ]:
!wget -O dt_table2.csv https://www.plus2net.com/sql_tutorial/dt_table2.php

Table with date and time

In [ ]:
!wget -O dt_table_tm.csv https://www.plus2net.com/sql_tutorial/dt_table_tm.php

Create Pandas Dataframe and print first 5 records

In [ ]:
import pandas as pd
df=pd.read_csv('dt_table.csv') # use your system path
print(df.head())

   record_id        date month-year
0       -370  2026-12-04   Dec-2026
1       -369  2026-12-03   Dec-2026
2       -368  2026-12-02   Dec-2026
3       -367  2026-12-01   Dec-2026
4       -366  2026-11-30   Nov-2026


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

dp = widgets.DatePicker(
    description='Pick a Date',
    disabled=False
)

output_date_picker = widgets.Output()

def on_date_change(change):
    with output_date_picker:
        clear_output()
        if change['name'] == 'value' and change['new'] is not None:
            print("Selected Date: ", change['new'])

dp.observe(on_date_change, names='value')

display(dp, output_date_picker)

DatePicker(value=None, description='Pick a Date')

Output()

Create one SQLite table using the dataframe.

In [ ]:
import sqlite3

# Name of the table within the SQLite database
table_name = 'dt_table'

# Create an SQLite database connection
my_conn = sqlite3.connect('my_db.db')

# Write the DataFrame to an SQLite table
df.to_sql(table_name, my_conn, if_exists='replace', index=False)

# Close the connection
my_conn.close()

Connect to database and select all records

In [ ]:
import sqlite3
my_path='my_db.db' #Change the path of your database
try:
    my_conn = sqlite3.connect(my_path) #
except sqlite3.Error as my_error:
  print("error: ",my_error)
else:
    print("Connected to database successfully")

try:
    query="SELECT * FROM dt_table LIMIT 0,5" # SQL to use for getting data from table
    r_set=my_conn.execute(query)  # execute the query using the connection
    for row in r_set:
        print(row)    # Print each row
except sqlite3.Error as my_error:
  print("error: ",my_error)

Connected to database successfully
(-370, '2026-12-04', 'Dec-2026')
(-369, '2026-12-03', 'Dec-2026')
(-368, '2026-12-02', 'Dec-2026')
(-367, '2026-12-01', 'Dec-2026')
(-366, '2026-11-30', 'Nov-2026')


Select today's records

In [ ]:
try:
    query="SELECT * FROM dt_table WHERE `date`=date('now')" #
    r_set=my_conn.execute(query)  # execute the query using the connection
    for row in r_set:
        print(row)    # Print each row
except sqlite3.Error as my_error:
  print("error: ",my_error)

(0, '2025-11-29', 'Nov-2025')


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import sqlite3
from datetime import datetime

In [ ]:
my_path = 'my_db.db'

# Task
Define the Python function `get_records_by_single_date` that takes a date string, connects to the SQLite database `my_db.db`, queries the `dt_table` for records matching that date, and prints them to an output.

## Define Get Records Function

### Subtask:
Define the Python function `get_records_by_single_date` that takes a date string, connects to the SQLite database, queries the `dt_table` for records matching that date, and prints them to an output. This function will be called by the date picker's event handler.


In [ ]:
def get_records_by_single_date(date_str):
    my_conn = None
    try:
        my_conn = sqlite3.connect('my_db.db')
        cursor = my_conn.cursor()
        query = "SELECT * FROM dt_table WHERE date = ?"
        cursor.execute(query, (date_str,))
        records = cursor.fetchall()

        if records:
            print(f"Records for date {date_str}:")
            for row in records:
                print(row)
        else:
            print(f"No records found for date {date_str}.")

    except sqlite3.Error as my_error:
        print(f"Database error: {my_error}")
    finally:
        if my_conn:
            my_conn.close()
            print("Database connection closed.")

**Reasoning**:
To implement the single date picker interface, I will create a DatePicker widget, an Output widget, and define an event handler to fetch and display records for the selected date using the previously defined `get_records_by_single_date` function.



In [ ]:
single_date_picker = widgets.DatePicker(
    description='Select Date',
    disabled=False,
    value=datetime.now().date()
)

output_single_date = widgets.Output()

def on_date_change_single(change):
    with output_single_date:
        output_single_date.clear_output()
        if change['name'] == 'value' and change['new'] is not None:
            selected_date = change['new'].strftime('%Y-%m-%d')
            get_records_by_single_date(selected_date)

single_date_picker.observe(on_date_change_single, names='value')

display(single_date_picker, output_single_date)

DatePicker(value=datetime.date(2025, 11, 27), description='Select Date')

Output()

In [ ]:
def get_records_by_date_range(start_date_str, end_date_str):
    my_conn = None
    try:
        my_conn = sqlite3.connect('my_db.db')
        cursor = my_conn.cursor()
        query = "SELECT * FROM dt_table WHERE date BETWEEN ? AND ?"
        cursor.execute(query, (start_date_str, end_date_str,))
        records = cursor.fetchall()

        if records:
            print(f"Records from {start_date_str} to {end_date_str}:")
            for row in records:
                print(row)
        else:
            print(f"No records found between {start_date_str} and {end_date_str}.")

    except sqlite3.Error as my_error:
        print(f"Database error: {my_error}")
    finally:
        if my_conn:
            my_conn.close()
            print("Database connection closed.")

In [ ]:
start_date_picker = widgets.DatePicker(
    description='Start Date',
    disabled=False,
    value=datetime.now().date()
)

end_date_picker = widgets.DatePicker(
    description='End Date',
    disabled=False,
    value=datetime.now().date()
)

query_button = widgets.Button(description="Query Records")
output_date_range = widgets.Output()

def on_button_click(b):
    with output_date_range:
        output_date_range.clear_output()
        start_date = start_date_picker.value
        end_date = end_date_picker.value
        if start_date and end_date:
            get_records_by_date_range(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))
        else:
            print("Please select both start and end dates.")

query_button.on_click(on_button_click)

display(start_date_picker, end_date_picker, query_button, output_date_range)

DatePicker(value=datetime.date(2025, 11, 27), description='Start Date')

DatePicker(value=datetime.date(2025, 11, 27), description='End Date')

Button(description='Query Records', style=ButtonStyle())

Output()